## Imports

In [1]:
!pip install openpyxl
!pip install PyArabic
!pip install git+https://github.com/ClaudeCoulombe/FrenchLefffLemmatizer.git &> /dev/null
!pip install emoji 
!pip install Arabic-Stopwords
!pip install tkseem
!pip install tnkeeh
!pip3 install fr-word-segment
!pip install pyspellchecker

In [2]:
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import numpy as np

import pandas as pd
from keras.preprocessing.text import Tokenizer

import nltk
import string
from french_lefff_lemmatizer.french_lefff_lemmatizer import FrenchLefffLemmatizer
from fastai.text.all import *

import sklearn
import regex as re
from unicodedata import normalize

import torch
from sklearn.model_selection import train_test_split
from keras.preprocessing.sequence import pad_sequences
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.transforms import ToTensor
from torch.autograd import Variable
from torch.optim.lr_scheduler import CyclicLR
from torchvision import models
from sklearn.metrics import accuracy_score, confusion_matrix,precision_score,recall_score,f1_score
import os
import gensim


# keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import GRU,MaxPooling1D,GlobalMaxPooling1D,Conv1D, Dense, Input, LSTM, Embedding, Dropout, Activation, Flatten, Bidirectional, GlobalMaxPool1D
from keras import callbacks
from keras.utils.vis_utils import plot_model

# sklearn
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer,CountVectorizer,TfidfTransformer 
from sklearn.metrics import roc_auc_score, accuracy_score,roc_curve, auc, plot_confusion_matrix, confusion_matrix
from sklearn.svm import LinearSVC
from sklearn import model_selection
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from tensorflow.keras.models import Sequential
from sklearn.manifold import TSNE
from sklearn.naive_bayes import MultinomialNB
from keras.preprocessing.text import Tokenizer
import emoji
from keras.models import Model
import seaborn as sn
import pyarabic.araby as ar
import tkseem as tk
import tnkeeh as tn
from nltk.stem.isri import ISRIStemmer
from spellchecker import SpellChecker
from wordsegment import load,segment
from keras.layers.merge import Concatenate
import tensorflow as tf
load()

## Loading Data

In [3]:
df_ar = pd.read_csv('/kaggle/input/twitter/ar_dataset.csv')
df_ar2 = pd.read_excel('/kaggle/input/twitter/arr.xlsx')

#main data
df_fr = pd.read_csv('/kaggle/input/twitter/fr_dataset.csv')
df_fr2 = pd.read_csv('/kaggle/input/twitter/french_tweets.csv')

## Exploring data

#### French Dataset

###### 1) Main dataset

In [4]:
df_fr.head()

In [5]:
df_fr.describe()

In [6]:
print('Size of the dataset:')
len(df_fr)

In [7]:
#test if the data contains null values
print('Nan value',df_fr.isnull().sum())

In [8]:
#take a look at the column of the dataframe to see the features
df_fr.columns

In [9]:
df_fr['sentiment'].value_counts()

In [10]:
#### Class distribution 
cum = df_fr['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='HITId',y='target',ax=ax)

###### 2) Assest dataset

In [11]:
df_fr2.head()

In [12]:
print('Size of the dataset:')
len(df_fr2)

In [13]:
#test if the data contains null values
print('Nan value',df_fr2.isnull().sum())

###### Note : 
in this dataset we don't have as much features as in the previous one!

#### Arabic dataset

###### 1) Main dataset

In [14]:
df_ar.head()

In [15]:
df_ar.describe()

In [16]:
print('Size of the dataset:')
len(df_ar)

In [17]:
#test if the data contains null values
print('Nan value',df_ar.isnull().sum())

In [18]:
df_ar['sentiment'].value_counts()

In [19]:
#### Class distribution 
cum = df_ar['target'].value_counts().to_frame()
cum['HITId'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='HITId',y='target',ax=ax)

###### 2) Assest dataset

In [20]:
df_ar2.head()

In [21]:
print('Size of the dataset:')
len(df_ar2)

In [22]:
#test if the data contains null values
print('Nan value',df_ar2.isnull().sum())

## Data processing

Since our task is a binary classification: negative or positive tweet, we will create a dataset that contains both negative and positive tweet. To do this, we will merge the previous dataset with the assest one that contains positive tweets. 

While merging, the assest doesn't contain the columns : HTDID, sentiment, directness, annotator_sentiment and group,some NaN values appeared. To avoid them these values will be replaced in each column with an appripriate value, for instance:


sentiment-> 'normal', 

directness->'direct',

groupe-> 'nothing' and 

annotator_sentiment->'indifference'

#### French Dataset

In [23]:
# negative tweets contained
# we put target = 0 negative
neg = df_fr.loc[df_fr['target']!='normal','target'] = 0
neg = df_fr
neg.head()

In [24]:
#positive tweets contained in the assest dataset
pos = df_fr2.loc[df_fr2['label']==1]
#since we have a several amount of data, we take a part of it
n = len(pos)
pos= pos[0:int(n/100)]
len(pos)
pos= pos.rename(columns={'label':'target','text':'tweet'})

In [25]:
# merging the two dataframes
data_fr = pd.concat([neg,pos], ignore_index=True, sort=False)
#print(len(data_fr))
data_fr.head()

In [26]:
# fixing nan values
data_fr.loc[data_fr['sentiment'].isnull().values==True,'sentiment'] = 'normal'
data_fr.loc[data_fr['directness'].isnull().values==True,'directness'] = 'direct'
data_fr.loc[data_fr['group'].isnull().values==True,'group'] = 'nothing'
data_fr.loc[data_fr['annotator_sentiment'].isnull().values==True,'annotator_sentiment'] = 'indifference'
data_fr = sklearn.utils.shuffle(data_fr)

In [27]:
#data_fr = data_fr.drop(columns='HITId')
data_fr.reindex().head()

In [28]:
plt.figure(figsize=(5,5))
plt.pie(data_fr["target"].value_counts(),labels=data_fr["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig("distribution des données dans les différentes classes.png")
plt.show()

In [29]:
#data_fr = oversample(data_fr)
data_fr = data_fr.drop(columns={'HITId'})
data_fr.head()

In [30]:
# distribution of classes: 0,1
cum = data_fr['target'].value_counts().to_frame()
cum['tweet'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='tweet',y='target',ax=ax)

In [31]:
plt.figure(figsize=(5,5))
plt.pie(data_fr["target"].value_counts(),labels=data_fr["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig("distribution des données dans les différentes classes.png")
plt.show()

#### Arabic dataset

In [32]:
neg = df_ar.loc[df_ar['target']!='normal','target'] = 0
neg = df_ar
neg.head()

In [33]:
df_ar2.head()

In [34]:
#positive tweets contained in the assest dataset
df_ar2.loc[df_ar2['Sentiment']=='Positive']
df_ar2.loc[df_ar2['Sentiment']=='Positive','Sentiment'] = 1
pos = df_ar2.loc[df_ar2["Sentiment"]==1]
pos= pos.rename(columns={'Sentiment':'target','Feed':'tweet'})
pos = pos.drop(columns={'ID'})

In [35]:
pos.head()

In [36]:
# merging the two dataframes
data_ar = pd.concat([neg[0:int(len(pos)/1.5)],pos], ignore_index=True, sort=False)
#print(len(data_ar))
data_ar.head()

In [37]:
data_ar.loc[data_ar['sentiment'].isnull().values==True,'sentiment'] = 'normal'
data_ar.loc[data_ar['directness'].isnull().values==True,'directness'] = 'direct'
data_ar.loc[data_ar['group'].isnull().values==True,'group'] = 'nothing'
data_ar.loc[data_ar['annotator_sentiment'].isnull().values==True,'annotator_sentiment'] = 'indifference'
data_ar = sklearn.utils.shuffle(data_ar)
data_ar.drop(columns='HITId')

In [38]:
plt.figure(figsize=(5,5))
plt.pie(data_ar["target"].value_counts(),labels=data_ar["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig(" ara distribution des données dans les différentes classes.png")
plt.show()

In [39]:
def resample(df):
    setnew = df
    lab0,lab1 = setnew[setnew['target'] == 0], setnew[setnew['target'] == 1]
    c0, c1 = setnew['target'].value_counts()
  
    lab0_sampled = lab0.sample(2*c0, replace=True) 
    lab1_sampled = lab1.sample(2*c1, replace=True)

    setnew_resampled = pd.concat([lab0_sampled,lab1_sampled], axis=0)

    return setnew_resampled

In [40]:
data_ar = resample(data_ar)
data_ar = data_ar.drop(columns={'HITId'})
data_ar.head()

In [41]:
# distribution of classes: 0,1
cum = data_ar['target'].value_counts().to_frame()
cum['tweet'] = cum.index
cumfig, ax = plt.subplots(figsize=(5,5))
sn.barplot(data=cum,x='tweet',y='target',ax=ax)

In [42]:
plt.figure(figsize=(5,5))
plt.pie(data_ar["target"].value_counts(),labels=data_ar["target"].value_counts().index,autopct=lambda p:f'{p:.2f}%',
        shadow=True,colors=['mediumvioletred','darkturquoise'],labeldistance = 1.1,textprops={'fontsize': 14})

plt.savefig(" ara distribution des données dans les différentes classes.png")
plt.show()

In [43]:
data_fr.columns

In [44]:
#encoding labels
le = preprocessing.LabelEncoder()
data_fr.sentiment = le.fit_transform(data_fr.sentiment)
data_fr.directness = le.fit_transform(data_fr.directness)
data_fr.annotator_sentiment = le.fit_transform(data_fr.annotator_sentiment)
data_fr.group = le.fit_transform(data_fr.group)
data_fr

In [45]:
data_ar.sentiment = le.fit_transform(data_ar.sentiment)
data_ar.directness = le.fit_transform(data_ar.directness)
data_ar.annotator_sentiment = le.fit_transform(data_ar.annotator_sentiment)
data_ar.group = le.fit_transform(data_ar.group)
data_ar.head()

In [46]:
def hash_fix(h):
    h1 = re.sub(r'[0-9]+', '', h)
    h2 = re.sub(r'#', '', h1)
    h3 = segment(str(h2))
    h4 = ' '.join(map(str, h3)) 
    return h4

In [47]:
tok_ar = tk.WordTokenizer()
tok_ar.train('/kaggle/input/twitter/ar_dataset.csv')
def prepro_ar(tweet):
    arabic_diacritics = re.compile(""" ّ    | # Tashdid
                             َ    | # Fatha
                             ً    | # Tanwin Fath
                             ُ    | # Damma
                             ٌ    | # Tanwin Damm
                             ِ    | # Kasra
                             ٍ    | # Tanwin Kasr
                             ْ    | # Sukun
                             ـ     # Tatwil/Kashida
                         """, re.VERBOSE)
    
    user = "*@user" # l'etoile se met au début, car l'arabe se lit de droite vers la gauche
    tweet = re.sub(arabic_diacritics, '', str(tweet))
    tweet = re.sub('r'+user, "", tweet)
    tweet = re.sub(r'(.)\1+', "", tweet) 
    tweet = ar.strip_tashkeel(tweet)
    tweet = ar.strip_tatweel(tweet)
    tweet = tweet.replace("@", " ")
    tweet = tweet.replace("_", " ")
    tweet = re.sub("ى", "ي", tweet)
    tweet = re.sub("ؤ", "ء", tweet)
    tweet = re.sub("ئ", "ء", tweet)
    tweet = re.sub("ة", "ه", tweet)
    tweet = re.sub("گ", "ك", tweet)
    tweet = tweet.replace("آ", "ا")
    tweet = tweet.replace("إ", "ا")
    tweet = tweet.replace("أ", "ا")
    tweet = tweet.replace("ؤ", "و")
    tweet = tweet.replace("ئ", "ي")
    tweet = nltk.tokenize.word_tokenize(tweet)
    tweet = [ISRIStemmer().suf32(w) for w in tweet]

    return tweet

In [48]:
l = "الله يعطيك ما تريد"
prepro_ar(l)

In [49]:
contractions_fr = {
        'administration':'admin',
        'avec':'ac',
        'beaucoup':'bp',
        'c’est-à-dire':'cad',
        'cependant':'cpd',
        'chose':'ch',
        'conclusion':'ccl',
        'confer ':'cf',
        'court terme':'ct',
        'dans':'ds',
        'dedans':'dd',
        'définition':'déf',
        'et cetera':'etc',
        'être':'ê',
        'exemple':'ex',
        'extérieur':'ext',
        'font':'ft',
        'général':'gal',
        'gouvernement':'gouv',
        'grand':'gd',
        'groupe':'gp',
        'identique':'idel',
        'introduction':'intro',
        'jour':'jr',
        'long terme':'lt',
        'lorsque':'lsq',
        'mais':'ms',
        'même':'^m',
        'moyen terme':'mt',
        'nombre':'nb',
        'nombreux':'nbx',
        'nombre':'nb',
        'nombreux':'nbx',
        'observation':'obs',
        'ordre du jour':'oj',
        'page':'p',
        'parce que':'pcq',
        'pendant':'pdt',
        'personne':'pers',
        'point':'pt',
        'peut-être':'pê',
        'pour':'pr',
        'pourtant':'prtt',
        'quand':'qd',
        'quantité':'qté',
        'que':'q',
        'quelqu’un':'qqn',
        'quelque chose':'qqch',
        'quelque':'qq',
        'quelquefois':'qqf',
        'question':'quest',
        'rendez-vous':'rdv',
        'responsabilité':'respité',
        'seulement':'slt',
        'solution':'sol',
        'sont':'st',
        'sous':'ss',
        'souvent':'svt',
        'temps':'tps',
        'toujours':'tjrs',
        'tous':'ts',
        'tout':'tt',
        'toute':'tte',
        'toutes':'ttes',
        'vous':'vs',
        'le':'l\'',
        'me':'m\'',
        'de':'d\'',
        'te':'t\'',
        'se':'s\'',
        'ce':'c\'',
        'ne':'n\'',
        'que':'qu\'',
        'jusque':'jusqu\'',
        'lorsque':'lorsqu\'',
        'puisque':'puisqu\'',
        'quelque':'quelqu\'',
        'quoique':'quoiqu\'',
        'parce que':'parce qu\'',
        'tel que':'tel qu\'',
        'telle que':'telle qu\'',       
        'faculte':'fac',
        'bien':'bin',    
        'attend':'att',
        'je': 'j\'',
        'rire':'ptdr',
        'rire':'lol',
        'rire':'lmfao',
}

In [50]:
french_stopwords = nltk.corpus.stopwords.words('french')
lemmatizer = FrenchLefffLemmatizer()
spell = SpellChecker(language='fr')

In [51]:
def prepro_fr(tweet):
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # normalize unicode characters
    tweet = normalize('NFD', tweet).encode('ascii','ignore')
    tweet = tweet.decode('UTF-8')
    #demojize
    tweet = emoji.demojize(tweet)
    if "#" in tweet:
        tweet = hash_fix(tweet)
    tweet = tweet.replace('user', '')
    tweet = tweet.replace('@user', '')
    tweet = tweet.replace('url', '')
    tweet = re.sub( r"(?i)\b((?:https?://|www\d{0,3}[.]|[a-z0-9.\-]+[.][a-z]{2,4}/)(?:[^\s()<>]+|\(([^\s()<>]+|(\([^\s()<>]+\)))*\))+(?:\(([^\s()<>]+|(\([^\s()<>]+\)))*\)|[^\s`!()\[\]{};:'\".,<>?«»“”‘’]))",'',tweet)
    # convert to lower case
    tweet = tweet.lower()
    tweet = tweet.replace('\'', '\' ') 
    # remove punctuation
    tweet = re.sub(r"\p{P}", lambda m: "-" if m.group(0) == "-" else "", tweet)
    # tokenization
    tweet = nltk.tokenize.word_tokenize(tweet)
    #contractions
    tweet = [list(contractions_fr.keys())[list(contractions_fr.values()).index(word)] if word in contractions_fr.values() else word for word in tweet]
    
    # stop words
    tweet = [w for w in tweet if w not in french_stopwords]
    sc = 'j[a-z]*'
    v = '[^aeyouisch]*'
    for w in tweet:
        x = re.findall(sc,w)
        xx = re.findall(v,w)
        if len(x)!=0:
            w = x[0].replace('j', 'je ')
            if len(x)>=3:
                w = w.replace(xx[0], '')
    # remove non-printable chars form each token
    tweet = [re_print.sub('', w) for w in tweet]
    # lemmatization
    tweet = [lemmatizer.lemmatize(w) for w in tweet]
    return tweet

In [52]:
data_ar.tweet = data_ar.tweet.apply(lambda t: prepro_ar(t))

In [53]:
data_ar

In [54]:
data_fr.tweet = data_fr.tweet.apply(lambda t: prepro_fr(t))

In [55]:
data_fr

In [56]:
tweets = data_fr.tweet.values

In [57]:
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.ar.300.vec.gz

In [58]:
!gunzip cc.ar.300.vec.gz


In [59]:
!wc -l cc.ar.300.vec

In [60]:
f_ar = open('cc.ar.300.vec', encoding='utf-8')

In [61]:
data_fr

In [62]:
data_ar.columns

In [63]:
labels_ar = data_ar.target.values
data_ar = data_ar.drop(columns={'target'})

In [64]:
X_train_ar,X_test_ar, y_train_ar,y_test_ar = train_test_split(data_ar,labels_ar,test_size=0.1)
# text ---> integer sequence
X_train_ar1 = []
X_test_ar1 = []

for t in X_train_ar.tweet:
    X_train_ar1.append(tok_ar.encode_sentences(t).flatten())
for t in X_test_ar.tweet:
    X_test_ar1.append(tok_ar.encode_sentences(t).flatten())

In [65]:
X_train_ar1 = pad_sequences(X_train_ar1, padding='post', maxlen=100)
X_test_ar1 = pad_sequences(X_test_ar1, padding='post', maxlen=100)

In [66]:
X_train_ar2 = X_train_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
X_test_ar2 = X_test_ar[['sentiment', 'directness', 'annotator_sentiment', 'group']].values

In [67]:
tweets_length_ar= [len(X_train_ar1[i]) for i in range(len(X_train_ar1))]
max_seq_len_ar = max(tweets_length_ar)
print(max_seq_len_ar)

In [68]:
#preparing embedding  model!
!wget https://dl.fbaipublicfiles.com/fasttext/vectors-crawl/cc.fr.300.vec.gz

In [69]:
!gunzip cc.fr.300.vec.gz

In [70]:
!wc -l cc.fr.300.vec

In [71]:
f_fr = open('cc.fr.300.vec', encoding='utf-8')

In [72]:
#loading pretrained model to word vecs
embeddings_index_fr = {}
for line in tqdm(f_fr):
    values = line.rstrip().rsplit(' ')
    word = values[0]
    vector = np.asarray(values[1:],'float32')
    embeddings_index_fr[word]=vector
f_fr.close()

print('found %s word vectors' % len(embeddings_index_fr))

In [73]:
labels_fr = data_fr.target.values
data_fr = data_fr.drop(columns={'target'})
X_train_fr,X_test_fr, y_train_fr,y_test_fr = train_test_split(data_fr,labels_fr,test_size=0.1)

In [74]:
X_train_fr1 = list(X_train_fr.tweet.values)
X_test_fr1 = list(X_test_fr.tweet.values)
tok_fr = Tokenizer()
tok_fr.fit_on_texts(data_fr.tweet.values)

X_train_fr1 = tok_fr.texts_to_sequences(X_train_fr1)
X_test_fr1 = tok_fr.texts_to_sequences(X_test_fr1)

In [75]:
X_train_fr1 = pad_sequences(X_train_fr1, maxlen=100)
X_test_fr1 = pad_sequences(X_test_fr1, maxlen=100)

In [76]:
X_train_fr2 = X_train_fr[['sentiment', 'directness', 'annotator_sentiment', 'group']].values
X_test_fr2 = X_test_fr[['sentiment', 'directness', 'annotator_sentiment', 'group']].values

In [77]:
#counting the length of the tweet and taking the max
tweets_length_fr= [len(X_train_fr1[i]) for i in range(len(X_train_fr1))]
max_seq_len_fr = max(tweets_length_fr)
print(max_seq_len_fr)

In [78]:
embed_dim = 300 

In [79]:
#embedding matrix
print('preparing embedding matrix...')
words_not_found_fr = []
word_index_fr = tok_fr.word_index
nb_words_fr = len(word_index_fr)+1
embedding_matrix_fr = np.zeros((nb_words_fr, embed_dim))
for word, i in word_index_fr.items():
    if i >= nb_words_fr:
        continue
    embedding_vector = embeddings_index_fr.get(word)
    if (embedding_vector is not None) and len(embedding_vector) > 0:
        # words not found in embedding index will be all-zeros.
        embedding_matrix_fr[i] = embedding_vector
    else:
        words_not_found_fr.append(word)
print('number of null word embeddings: %d' % np.sum(np.sum(embedding_matrix_fr, axis=1) == 0))
print(len(embedding_matrix_fr))

In [83]:
input_fr_1 = Input(shape=(max_seq_len_fr,))
input_ar_1 = Input(shape=(max_seq_len_ar,))
input_2 = Input(shape=(4,))

In [84]:
embedding_layer_fr = Embedding(
                            nb_words_fr, 
                            embed_dim,  
                            weights=[embedding_matrix_fr],
                            input_length=max_seq_len_fr,
                            trainable=False
                    )(input_fr_1)

In [85]:
def NN(lang,input_1,input_2):
    model = Sequential()
    if lang=='AR':  
        embedding_layer = Embedding(input_dim = tok_ar.vocab_size,output_dim = embed_dim, input_length=max_seq_len_ar)(input_ar_1)
        #model.add(embedding_layer)
    else:
        embedding_layer = embedding_layer_fr
        #model.add(embedding_layer_fr)
    # 1st bi-LSTM layer
    bi_lstm = Bidirectional(LSTM(128,return_sequences=True))(embedding_layer)
    bilstm_dropout = Dropout(0.5)(bi_lstm)
    #x = GlobalMaxPool1D()(bilstm_dropout)
    #FC layers
    dense_layer_1 = Dense(100, activation='relu')(bilstm_dropout)
    #model.add(dense_layer_1)
    dense_layer_2 = Dense(10, activation='relu')(dense_layer_1)
    #model.add(Dense(64, activation='relu')(dense_layer_1))
    #model.add(Dropout(0.5))
    
    concat_layer = Concatenate()([bilstm_dropout, dense_layer_2])
    dense_layer_3 = Dense(10, activation='relu')(concat_layer)
    output = Dense(1, activation='sigmoid')(dense_layer_3)
    #model.add(Dense(1, activation='sigmoid'))
    model = Model(inputs=[input_1, input_2], outputs=output)
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    plot_model(model, to_file='model.png', show_shapes=True, show_layer_names=True)
    return model

In [87]:
early = callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')
num_epochs = 10
batch_size = 128
model_fr = NN('FR',input_fr_1,input_2)
model_ar = NN('AR',input_ar_1,input_2)

In [88]:
print('Arabic:')

Xar = [np.asarray(X_train_ar1), np.asarray(X_train_ar2)]
history_ar = model_ar.fit(Xar,
                          y_train_ar.astype(np.float32),
                          batch_size=batch_size,
                          epochs=num_epochs,
                          validation_split=0.15,
                          callbacks = [early],
                          verbose=1)

In [89]:
print('French:')
Xfr = [X_train_fr1,X_train_fr2]
history_fr = model_fr.fit(Xfr,
                          y_train_fr.astype(np.float32),
                          batch_size=batch_size,
                          epochs=num_epochs,
                          validation_split=0.15,
                          callbacks = [early],
                          verbose=1)

In [90]:
def plot_acc_loss(history):
    """
    Plot accuracy and loss of a model
    @params:
            - history: history of the model
    @return:
            plots
    """
    fig,ax = plt.subplots(1,2,figsize=(10,5))
    l = list(history.history.keys())
    print(l)
    # accuracy plot
    ax[0].plot(history.history[l[1]])
    ax[0].plot(history.history[l[3]])
    ax[0].set_title('model accuracy')
    ax[0].set_ylabel('accuracy')
    ax[0].set_xlabel('epoch')
    ax[0].legend(['train', 'test'], loc='upper left')
    # loss plot
    ax[1].plot(history.history[l[0]])
    ax[1].plot(history.history[l[2]])
    ax[1].set_title('model loss')
    ax[1].set_ylabel('loss')
    ax[1].set_xlabel('epoch')
    ax[1].legend(['train', 'test'], loc='upper left')

In [91]:
plot_acc_loss(history_ar)

In [92]:
plot_acc_loss(history_fr)

In [105]:
Xtestfr = [np.asarray(X_test_fr1), np.asarray(X_test_fr2)]
test_loss, test_acc = model_fr.evaluate(Xtestfr,y_test_fr.astype(np.float32))
print('Test accuracy:', test_acc)

In [95]:
def predicted_label(model,x):
    pred = model.predict(x)
    lab_pred = []
    for i in range(len(pred)):
        if pred[i][0][0]>=0.5:
            lab_pred.append(1)
        else:
            lab_pred.append(0)
    return lab_pred

In [96]:
def plot_cm(model,x,y):
    pred = model.predict(x)
    y_pred = predicted_label(model,x)
    cm = confusion_matrix(list(y),y_pred)  
    sn.heatmap(cm, annot=True)

In [97]:
print("Confusion Matrix for french dataset")
Xtestfr = [np.asarray(X_test_fr1), np.asarray(X_test_fr2)]
plot_cm(model_fr,Xtestfr,y_test_fr)

In [98]:
print("Confusion Matrix for arabic dataset")
Xtestar = [np.asarray(X_test_ar1), np.asarray(X_test_ar2)]
plot_cm(model_ar,Xtestar,y_test_ar)

In [99]:
pred_ar =predicted_label(model_ar,Xtestar)
# accuracy: (tp + tn) / (p + n)
accuracy_ar = accuracy_score(list(y_test_ar), pred_ar)
print('Accuracy: %f' % accuracy_ar)
# precision tp / (tp + fp)
precision_ar = precision_score(list(y_test_ar), pred_ar)
print('Precision: %f' % precision_ar)
# recall: tp / (tp + fn)
recall_ar = recall_score(list(y_test_ar), pred_ar)
print('Recall: %f' % recall_ar)
# f1: 2 tp / (2 tp + fp + fn)
f1_ar = f1_score(list(y_test_ar), pred_ar)
print('F1 score: %f' % f1_ar)


In [109]:
pred_fr =predicted_label(model_fr,Xtestfr)
# accuracy: (tp + tn) / (p + n)
accuracy_fr = accuracy_score(list(y_test_fr), pred_fr)
print('Accuracy: %f' % accuracy_fr)
# precision tp / (tp + fp)
precision_fr = precision_score(list(y_test_fr), pred_fr)
print('Precision: %f' % precision_fr)
# recall: tp / (tp + fn)
recall_fr = recall_score(list(y_test_fr), pred_fr)

print('Recall: %f' % recall_fr)
# f1: 2 tp / (2 tp + fp + fn)
f1_fr = f1_score(list(y_test_fr), pred_fr)
print('F1 score: %f' % f1_fr)
